## Calibração de intrísicos usando chessboard oii

- Utiliza a biblioteca OpenCV para obter a matriz de intrísicos da câmera
- Para obtê-la, fazemos uma comparação com distâncias entre pontos conhecidas no mundo real e as distâncias entre pontos no plano da imagem
- É escolhido um plano de calibração no qual conhecemos as distâncias entre os pontos, como por exemplo entre os cantos dos quadrados de um tabuleiro de xadrez

- Para realizar a calibração, é preciso tirar fotos do plano em diferentes poses
- Algumas boas práticas são:
    - Escolha um ambiente iluminado e, se possível, utilize uma fonte luminosa atrás da câmera
    - Prepare um dataset com várias imagens, assim você pode escolher as melhores para realizar a calibração
    - O plano de calibração precisa cobrir a maior parte das fotos, por isso se atente no tamanho do plano que você está utilizando

### Passo a passo da calibração
- Antes de tirar as fotos você precisa de um plano de calibração 
    - Para criar o seu, use o [código](https://docs.opencv.org/4.x/da/d0d/tutorial_camera_calibration_pattern.html) disponibilizado pelo próprio OpenCV
- As imagens utilizadas na calibração precisam estar em escala de cinza!
    - Antes de analisar as imagens, converta-as para grayscale 

- Para encontrar os cantos dos quadrados do tabuleiro, são usadas duas abordagens:
    - A primeira utiliza um método que determina o canto do quadrado no ponto em que o branco se torna preto na imagem, pela função `findChessboardCorners()`. Os cantos encontrados não são tão exatos, por isso passam por um refinamento pela função `cornerSubPix()`
    - A segunda utiliza uma abordagem diferente, na qual usa a função `findChessboardCornersSB()`

### Importação das bibliotecas e informações sobre o padrão de calibração

A depender do padrão que escolher, você deverá mudar:
- patternSize (colunas X linhas)
- squareSize (tamanho do quadrado em mm)
- imgSize (tamanho da imagem)

In [1]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

patternSize = (10,7)
squareSize = 30
imgSize = (2592,1944) 

# USE THE DIRECTORY WHERE YOUR IMAGES ARE! 
images = glob.glob("*jpg")

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

## Definir função para reconstrução dos pontos
- Para encontrar os pontos dos cantos do tabuleiro no mundo real, utiliza-se o tamanho dos quadrados e a quantidade de linhas e colunas do tabuleiro

In [2]:
def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize
    return X

boardPoints = construct3DPoints(patternSize,squareSize)
worldPoints = []
imagePoints = []
worldPointsSB = []
imagePointsSB = [] 

### Abordagem com `findChessboardCorners()` e `cornerSubPix()`

Caso queira ver os cantos dectados pela função, utilize a função `drawChessboardCorners()`

In [5]:
counter = 0
for fname in images:
    print("=> Processing image {0}".format(fname))
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, patternSize, None)
    if ret == True:
        print("Corners found!")
        cornersRefined = cv2.cornerSubPix(gray, corners, (7,7), (-1,-1), criteria)
        imagePoints.append(cornersRefined)
        worldPoints.append(boardPoints)
        counter+=1
        cv2.drawChessboardCorners(img, patternSize, cornersRefined, ret)
        cv2.imshow('img', img)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()


=> Processing image chess1.jpg
Corners found!
=> Processing image chess10.jpg
Corners found!
=> Processing image chess11.jpg
Corners found!
=> Processing image chess12.jpg
Corners found!
=> Processing image chess13.jpg
Corners found!
=> Processing image chess14.jpg
Corners found!
=> Processing image chess15.jpg
Corners found!
=> Processing image chess16.jpg
Corners found!
=> Processing image chess17.jpg
Corners found!
=> Processing image chess18.jpg
Corners found!
=> Processing image chess19.jpg
Corners found!
=> Processing image chess2.jpg
Corners found!
=> Processing image chess20.jpg
Corners found!
=> Processing image chess3.jpg
Corners found!
=> Processing image chess4.jpg
Corners found!
=> Processing image chess5.jpg
Corners found!
=> Processing image chess6.jpg
Corners found!
=> Processing image chess7.jpg
Corners found!
=> Processing image chess8.jpg
Corners found!
=> Processing image chess9.jpg
Corners found!


In [4]:
imagePoints

[array([[[ 864.64886,  814.40906]],
 
        [[ 939.84973,  819.56726]],
 
        [[1015.82935,  824.3444 ]],
 
        [[1092.6824 ,  830.3452 ]],
 
        [[1168.9386 ,  836.2941 ]],
 
        [[1245.278  ,  841.4621 ]],
 
        [[1321.649  ,  847.4511 ]],
 
        [[1398.4387 ,  852.51   ]],
 
        [[1474.3916 ,  857.7908 ]],
 
        [[1550.429  ,  863.5856 ]],
 
        [[ 857.55554,  888.88336]],
 
        [[ 933.4757 ,  894.328  ]],
 
        [[1009.47363,  899.32336]],
 
        [[1086.7516 ,  904.5658 ]],
 
        [[1163.5706 ,  910.98724]],
 
        [[1240.5619 ,  916.5081 ]],
 
        [[1317.4458 ,  922.1969 ]],
 
        [[1394.5613 ,  927.335  ]],
 
        [[1470.9374 ,  932.3713 ]],
 
        [[1547.2938 ,  938.0736 ]],
 
        [[ 850.2908 ,  964.357  ]],
 
        [[ 927.3908 ,  969.9243 ]],
 
        [[1003.84406,  975.1668 ]],
 
        [[1081.1378 ,  980.65314]],
 
        [[1158.575  ,  986.75946]],
 
        [[1235.796  ,  992.24005]],
 
        [[13

### Calibração da câmera

In [6]:
flagsCalib = cv2.CALIB_RATIONAL_MODEL 

initialCameraMatrix = np.array([[ 1000.,    0., imgSize[0]/2.],
                                 [    0., 1000., imgSize[1]/2.],
                                 [    0.,    0.,           1.]])

initialDistCoeffs = np.zeros((5,1))


ret, cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsIntrinsics, stdDeviationsExtrinsics, perViewError = cv2.calibrateCameraExtended(worldPoints, imagePoints, imgSize, initialCameraMatrix, initialDistCoeffs,flags=flagsCalib)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", distCoeffs)

Using 20 of 20 images
RMS re-projection error: 1.1171705526748483
Camera Matrix:
 [[1.03860826e+04 0.00000000e+00 1.28818478e+03]
 [0.00000000e+00 1.18423694e+04 9.38530455e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:
 [[-2.88559361e+00]
 [ 1.34738929e+02]
 [ 1.12473019e-01]
 [-3.25544297e-02]
 [-4.88317877e+03]
 [ 2.97659549e+00]
 [-1.06433272e+02]
 [ 4.44416013e+03]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


### Erro de reprojeção para cada imagem

In [12]:
print("RE-PROJECTION ERROR OF EACH VIEW - CHESSBOARD\n",perViewError)

RE-PROJECTION ERROR OF EACH VIEW - CHESSBOARD
 [[0.28667768]
 [0.19690801]
 [0.33870449]
 [0.30189558]
 [0.30732638]
 [0.22069464]]


### Abordagem com `findChessboardCornersSB()`

Caso queira ver os cantos dectados pela função, utilize a função `drawChessboardCorners()`

In [3]:
counter = 0
for fname in images:
    print("=> Processing image {0}".format(fname))
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, cornersSB = cv2.findChessboardCornersSB(gray, patternSize, cv2.CALIB_CB_EXHAUSTIVE + cv2.CALIB_CB_ACCURACY + cv2.CALIB_CB_NORMALIZE_IMAGE)
    if ret == True:
        print("Corners found!")
        imagePointsSB.append(cornersSB)
        worldPointsSB.append(boardPoints)
        counter+=1
        #cv2.drawChessboardCorners(img, patternSize, cornersSB, ret)
        #cv2.imshow('img', img)
        #cv2.waitKey(1000)
        #cv2.destroyAllWindows()


=> Processing image chess1.jpg
Corners found!
=> Processing image chess10.jpg
Corners found!
=> Processing image chess11.jpg
Corners found!
=> Processing image chess12.jpg
Corners found!
=> Processing image chess13.jpg
Corners found!
=> Processing image chess14.jpg
Corners found!
=> Processing image chess15.jpg
Corners found!
=> Processing image chess16.jpg
Corners found!
=> Processing image chess17.jpg
Corners found!
=> Processing image chess18.jpg
Corners found!
=> Processing image chess19.jpg
Corners found!
=> Processing image chess2.jpg
Corners found!
=> Processing image chess20.jpg
Corners found!
=> Processing image chess3.jpg
Corners found!
=> Processing image chess4.jpg
Corners found!
=> Processing image chess5.jpg
Corners found!
=> Processing image chess6.jpg
Corners found!
=> Processing image chess7.jpg
Corners found!
=> Processing image chess8.jpg
Corners found!
=> Processing image chess9.jpg
Corners found!


### Calibração da câmera

In [8]:
#adicionar novas flags!!!
flagsCalib = cv2.CALIB_RATIONAL_MODEL

initialCameraMatrix = np.array([[ 1000.,    0., imgSize[0]/2.],
                                 [    0., 1000., imgSize[1]/2.],
                                 [    0.,    0.,           1.]])

initialDistCoeffs = np.zeros((5,1))


ret, cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsIntrinsics, stdDeviationsExtrinsics, perViewErrorSB = cv2.calibrateCameraExtended(worldPointsSB, imagePointsSB, imgSize, initialCameraMatrix, initialDistCoeffs,flags=flagsCalib)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", distCoeffs)

Using 20 of 20 images
RMS re-projection error: 1.1179794888354586
Camera Matrix:
 [[1.15853072e+04 0.00000000e+00 1.25729295e+03]
 [0.00000000e+00 1.49642235e+04 9.68492952e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:
 [[-2.43147959e+00]
 [-1.31942916e+03]
 [ 9.41026164e-02]
 [-3.17982802e-02]
 [-8.65457881e+00]
 [ 4.02339028e+00]
 [-1.51832492e+03]
 [ 7.85781029e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


In [4]:
flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)
ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPointsSB), -2), imagePointsSB, imgSize, None, None,flags=flagsCalib,criteria=calibrateCriteria)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", k)

Using 20 of 20 images
RMS re-projection error: 0.4773933292374432
Camera Matrix:
 [[3.09490979e+03 0.00000000e+00 1.35833252e+03]
 [0.00000000e+00 3.10053161e+03 1.14352179e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:
 [[-0.18814418]
 [ 0.04370439]
 [ 0.74739738]
 [-5.20614682]]


### Erro de reprojeção para cada imagem

In [9]:
stdDeviationsIntrinsics

array([[6.17433349e+00],
       [6.21725736e+00],
       [1.98891782e+00],
       [1.67545998e+00],
       [8.81126905e+01],
       [2.50880516e+02],
       [6.70817273e-04],
       [7.50000474e-04],
       [6.73181149e+02],
       [8.80404489e+01],
       [2.57223552e+02],
       [6.83139953e+02],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00]])

In [10]:
print("RE-PROJECTION ERROR OF EACH VIEW - CHESSBOARD\n",perViewErrorSB)

RE-PROJECTION ERROR OF EACH VIEW - CHESSBOARD
 [[0.27921098]
 [0.18899875]
 [0.32593553]
 [0.28575809]
 [0.28365508]
 [0.21192884]]


### Removendo distorção de uma imagem

Escolhe-se uma imagem qualquer do plano de calibração e com a matriz K e os coeficientes de distorção encontrados na calibração é possível remover as distorções da foto

In [7]:
def undistortFisheye(imgpath,K,D,DIM,axis,method,scale,corners):
    img = cv2.imread(imgpath)
    '''new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, DIM, np.eye(3), balance=0)

    new_K = K.copy()
    new_K[0,0]=K[0,0]*scale
    new_K[1,1]=K[1,1]*scale'''
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    fx = K[0,0]
    fy = K[1,1]
    cx = K[0,2]
    cy = K[1,2]
    undCorners_norm = cv2.fisheye.undistortPoints(corners, K, D)
    undCorners_norm = undCorners_norm.reshape(-1,2)
    undistCorners = np.zeros_like(undCorners_norm)
    for i, (x, y) in enumerate(undCorners_norm):
        px = x*fx + cx
        py = y*fy + cy
        undistCorners[i,0] = px
        undistCorners[i,1] = py    
    cv2.drawChessboardCorners(undistorted_img, patternSize, undistCorners, _)

    axs[axis].imshow(undistorted_img[:,:,::-1])
    axs[axis].axis('off')
    axs[axis].set_title('undistort '+method)
    #cv2.imwrite('undistort'+method+'.png', undistorted_img)

    return corners,undistCorners
image = cv2.imread("chess09.jpg")
undistortFisheye(image,cameraMatrix,k,imgSize,1,'openCV',1,cornersSB)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:668: error: (-215:Assertion failed) !ssize.empty() in function 'cv::remapBilinear'


In [6]:
image = cv2.imread("chess09.jpg")

newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, k, imgSize, 1, imgSize)
undistortedImage = cv2.undistort(image, cameraMatrix, k, None, newCameraMatrix)

ig, axs = plt.subplots(1,2,figsize=(7, 4), dpi=500)
axs[0].imshow(image[:,:,::-1])
axs[0].axis('off')
axs[0].set_title('distorted')
axs[1].imshow(undistortedImage[:,:,::-1])
axs[1].axis('off')
axs[1].set_title('undistorted')

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\undistort.dispatch.cpp:298: error: (-215:Assertion failed) dst.data != src.data in function 'cv::undistort'
